In [576]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint

# DATE = "2022-12-14"
# URL = "https://hansard.parliament.uk/Commons/2022-12-14/debates/22121461000004/Engagements"

# DATE = "2022-12-07"
# URL = "https://hansard.parliament.uk/Commons/2022-12-07/debates/8253154F-B5B4-4507-8EDD-ACB8A3267B4C/Engagements"

# DATE = "2022-11-30"
# URL = "https://hansard.parliament.uk/Commons/2022-11-30/debates/1BFA1A26-3814-40E6-B310-418F6C03ED30/Engagements"

In [577]:
import urllib.request

# TODO: Understand
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

url = URL
headers={'User-Agent':user_agent,}

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read().decode(response.headers.get_content_charset())

In [557]:
soup = BeautifulSoup(data, 'html.parser')

In [558]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="/dist/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="https://cookiemanager.parliament.uk/cookiemanager.css" rel="stylesheet">
   <!-- Google Tag Manager -->
   <script>
    (function (w, d, s, l, i) {
            w[l] = w[l] || [];

            w[l].push({
                'gtm.start': new Date().getTime(),
                event: 'gtm.js'
            });

            var f = d.getElementsByTagName(s)[0],
                j = d.createElement(s),
                dl = l != 'dataLayer' ? '&l=' + l : '';

            j.async = true;
            j.src = 'https://www.googletagmanager.com/gtm.js?id=' + i + dl;
            f.parentNode.insertBefore(j, f);
        })(window, document, 'script', 'dataLayer', 'GTM-PWZPG2V');
   </script>
   <!-- End Google Tag Manager -->
   <title>
    E

In [559]:
import re
members = soup.find_all("a", class_="attributed-to-details with-link")

member_ids = []
for _ in members:
    id_ = re.findall(r"[0-9]+", _["href"])[0]

    # Exclude the Speaker
    if id_ == "467":
        continue

    else: member_ids.append(id_)

len(member_ids)
member_ids

['4453',
 '4483',
 '4453',
 '4483',
 '103',
 '4483',
 '4514',
 '4483',
 '4514',
 '4483',
 '4514',
 '4483',
 '4514',
 '4483',
 '4514',
 '4483',
 '4514',
 '4483',
 '3958',
 '4483',
 '4390',
 '4483',
 '4390',
 '4483',
 '4052',
 '4483',
 '4846',
 '4483',
 '4840',
 '4483',
 '4820',
 '4483',
 '4050',
 '4483',
 '4269',
 '4483',
 '4084',
 '4483',
 '4776',
 '4483',
 '4459',
 '4483',
 '4523',
 '4483',
 '3939',
 '4483',
 '4378',
 '4483',
 '4792',
 '4483',
 '4413',
 '4483',
 '4803',
 '4483',
 '4470',
 '4483']

In [560]:
import re

# Search all Debate 'Contributions'
debates = soup.find_all("div", class_="contribution")

pmqs = []
pm = []

question_debate = []
counta = 0

for tag in debates:
    if tag.attrs["data-share-title"] == "Contribution by Mr Speaker":
        continue

    if tag.attrs["data-share-title"] == "Contribution by The Prime Minister (Rishi Sunak)" or tag.attrs["data-share-title"] == "Contribution by The Prime Minister":
        member = "Prime Minister"
        children = tag.findChildren()
        for child in children:
            if child.attrs == {"class": ['content']}:
                pmqs.append((member, child.text.strip("\n")))

    else:
        member = re.sub(r"Contribution by ", "", tag.attrs["data-share-title"])
        children = tag.findChildren()


        for child in children:
            if child.attrs == {"class": ['content']}:
                pmqs.append((member, child.text.strip("\n")))

In [561]:
len(pmqs), pmqs

(56,
 [('Tommy Sheppard (Edinburgh East) (SNP)',
   'Q1.  If he will list his official engagements for Wednesday 30 November. (902522)'),
  ('Prime Minister',
   'I am sure that colleagues around the House will want to join me in congratulating England on last night, in commending Wales for inspiring millions and in wishing everyone a happy St Andrew’s day.This morning, I had meetings with ministerial colleagues and others. In addition to my duties in this House, I shall have further such meetings later today.'),
  ('Tommy Sheppard',
   'In 2014, the Prime Minister’s predecessor David Cameron signed up to the Smith commission, which promised among other things that“nothing in this report prevents Scotland becoming an independent country…should the people of Scotland so choose.”Does the Prime Minister share that view? If he does, in the light of last week’s Supreme Court judgment, will he bring forward legislation to allow that choice to be exercised?'),
  ('Prime Minister',
   'We did 

In [562]:
# Create Member Profiles
def get_member(id_):
    members_api = f"https://members-api.parliament.uk/api/Members/{id_}"

    try:
        result = requests.get(members_api, json=True).json()
    except:
        return None

    if len(result["value"]) > 0:
        title = result["value"]["nameFullTitle"]
        party = result["value"]["latestParty"]["name"]
        img = result["value"]["thumbnailUrl"]

        return id_, title, party, img

member_profiles = []
for id_ in member_ids:
    member_profiles.append(get_member(id_))

In [563]:
member_profiles

[('4453',
  'Tommy Sheppard MP',
  'Scottish National Party',
  'https://members-api.parliament.uk/api/Members/4453/Thumbnail'),
 ('4483',
  'Rt Hon Rishi Sunak MP',
  'Conservative',
  'https://members-api.parliament.uk/api/Members/4483/Thumbnail'),
 ('4453',
  'Tommy Sheppard MP',
  'Scottish National Party',
  'https://members-api.parliament.uk/api/Members/4453/Thumbnail'),
 ('4483',
  'Rt Hon Rishi Sunak MP',
  'Conservative',
  'https://members-api.parliament.uk/api/Members/4483/Thumbnail'),
 ('103',
  'Sir Paul Beresford MP',
  'Conservative',
  'https://members-api.parliament.uk/api/Members/103/Thumbnail'),
 ('4483',
  'Rt Hon Rishi Sunak MP',
  'Conservative',
  'https://members-api.parliament.uk/api/Members/4483/Thumbnail'),
 ('4514',
  'Rt Hon Keir Starmer MP',
  'Labour',
  'https://members-api.parliament.uk/api/Members/4514/Thumbnail'),
 ('4483',
  'Rt Hon Rishi Sunak MP',
  'Conservative',
  'https://members-api.parliament.uk/api/Members/4483/Thumbnail'),
 ('4514',
  'Rt H

In [564]:
# Asset Member Profiles == Debate Items in PMQs
assert len(member_profiles) == len(pmqs)

In [565]:
# Determine PMQ Boundaries
pm_id = "4483"
history = []
boundaries = []
start_idx = 0
_ = 0


speakers = [_ for _ in member_profiles if _[0] != pm_id]
last_member_id = speakers[-1][0]

while _ < len(member_profiles):
    cur_member_id = member_profiles[_][0]
    cur_member = member_profiles[_][1]

    if cur_member_id == pm_id:
        _ += 1
        continue

    history.append(cur_member_id)
    previous_member_id = history[-2] if len(history) > 1 else history[-1]

    if cur_member_id != previous_member_id:
        boundaries.append([start_idx, _ - 1])
        start_idx = _

    if cur_member_id == last_member_id:
        boundaries.append([start_idx, len(pmqs)])

    _ += 1

len(boundaries)
boundaries

[[0, 3],
 [4, 5],
 [6, 17],
 [18, 19],
 [20, 23],
 [24, 25],
 [26, 27],
 [28, 29],
 [30, 31],
 [32, 33],
 [34, 35],
 [36, 37],
 [38, 39],
 [40, 41],
 [42, 43],
 [44, 45],
 [46, 47],
 [48, 49],
 [50, 51],
 [52, 53],
 [54, 56]]

In [566]:
# Chunk into PMQ Debate Items
debate_chunks = []
for _ in boundaries:
    seek, stop = _[0], _[1]+1
    debate_chunks.append((seek, stop, pmqs[seek: stop]))

debate = (_ for _ in debate_chunks)

next(debate)

(0,
 4,
 [('Tommy Sheppard (Edinburgh East) (SNP)',
   'Q1.  If he will list his official engagements for Wednesday 30 November. (902522)'),
  ('Prime Minister',
   'I am sure that colleagues around the House will want to join me in congratulating England on last night, in commending Wales for inspiring millions and in wishing everyone a happy St Andrew’s day.This morning, I had meetings with ministerial colleagues and others. In addition to my duties in this House, I shall have further such meetings later today.'),
  ('Tommy Sheppard',
   'In 2014, the Prime Minister’s predecessor David Cameron signed up to the Smith commission, which promised among other things that“nothing in this report prevents Scotland becoming an independent country…should the people of Scotland so choose.”Does the Prime Minister share that view? If he does, in the light of last week’s Supreme Court judgment, will he bring forward legislation to allow that choice to be exercised?'),
  ('Prime Minister',
   'We d

In [567]:
# Iterate PMQ Debate Chunk
next(debate)

(4,
 6,
 [('Sir Paul Beresford (Mole Valley) (Con)',
   'Q3.  I have just returned from the south Pacific and—[Interruption.] I have resisted the All Blacks jokes. I found a deep concern there about the expanding tentacles of communist China. Does my right hon. Friend agree that China is more than just “a systemic challenge”, as he has put it, and is in fact an expanding, serious geopolitical threat? (902524)'),
  ('Prime Minister',
   'China is indeed a country with fundamentally different values from ours and an authoritarian leadership intent on reshaping the international order, but actions speak louder than words. That is why we passed the National Security and Investment Act 2021. Just recently, we used that Act to block the sale of Newport Wafer Fab, and this week, with our announcement of Sizewell C, we ensured that China’s state-owned nuclear energy corporation will no longer be a part of the project. This Government are making sure that we protect our country’s security.')])

In [568]:
# PMQ Storage Objects
class DebateItem:
    def __init__(self, date):
        self.date = date
        self.debate_idx = ""
        self.member = {}
        self.question = {
            "idx": []
        }
        self.response = {}

debate_item = DebateItem(DATE).__dict__

class PMQ:
    def __init__(self, type):
        self.type = type

    def __call__(self, debate_item):
        if self.type == "opposition":
            debate_item["question"]["opposition"] = []
            debate_item["response"]["opposition"] = []

            return debate_item

        elif self.type == "member":
            debate_item["question"]["member"] = []
            debate_item["response"]["member"] = []

            return debate_item

        else:
            raise ValueError("incorrect type")

class Member:
    def __init__(self, id, title, party, img_url):
        self.id = id
        self.title = title
        self.party = party
        self.img_url = img_url

pmq = PMQ(type="member")(debate_item)
pprint(pmq)

{'date': '2022-11-30',
 'debate_idx': '',
 'member': {},
 'question': {'idx': [], 'member': []},
 'response': {'member': []}}


In [569]:
# Run Debate
import re

debate_chunks = []
for _ in boundaries:
    seek, stop = _[0], _[1]+1
    debate_chunks.append((seek, stop, pmqs[seek: stop]))

def get_type(member_id):
    """ Get PMQ Type == Member | Opposition """
    if member_id in ["4514", "4390", "4735"]:
        return "opposition"

    else: return "member"

def clean(question):
    """ Clean question index-reference from debate item """
    cleaned = re.sub(r"(\b[Q]{1}[0-9]+[.])", "", question)
    cleaned = cleaned.lstrip()

    return cleaned

""" Construct generator object to iterate Debate Chunks """
debate = (_ for _ in debate_chunks)

pmq_items = []
caught_counta = 0
counta = 0

while counta < (len(debate_chunks)):

    seek, stop, discourse = next(debate)
    member_id, member_title, member_party, member_img = member_profiles[seek]

    member = Member(id=member_id, title=member_title, party=member_party, img_url=member_img).__dict__

    # TODO: Make 'Prime Minister' Robust
    questions = [_ for _ in discourse if _[0] != "Prime Minister"]
    responses = [_ for _ in discourse if _[0] == "Prime Minister"]

    type = get_type(member_id)

    debate_item = DebateItem(date=DATE).__dict__
    pmq = PMQ(type=type)(debate_item)

    pmq["debate_idx"] = counta + 1

    for idx, _ in enumerate(zip(questions, responses)):
        question, response = _

        q = question[1]
        r = response[1]

        tabled = re.findall(r"(\b[Q]{1}[0-9]+[.])", q)
        tabled = str(tabled[0]) if tabled != [] else False

        q = clean(question[1])

        if idx == 0:
            pmq["member"] = member

        if type == "opposition":
            q_idx = f"OP.{idx + 1}"

            pmq["question"]["idx"].append(q_idx)
            pmq["question"]["opposition"].append(q)
            pmq["response"]["opposition"].append(r)

        else:
            if idx > 0:
                """CASE: Supplement """
                prev_idx = pmq["question"]["idx"][-1]
                q_idx = f"{prev_idx}{idx}"

            elif not tabled:
                """CASE: Caught """
                caught_counta += 1
                q_idx = f"CAUGHT.{caught_counta}"

            else: q_idx = tabled

            pmq["question"]["idx"].append(q_idx)
            pmq["question"]["member"].append(q)
            pmq["response"]["member"].append(r)

    pmq_items.append(pmq)
    counta += 1

In [570]:
pprint(pmq_items)

[{'date': '2022-11-30',
  'debate_idx': 1,
  'member': {'id': '4453',
             'img_url': 'https://members-api.parliament.uk/api/Members/4453/Thumbnail',
             'party': 'Scottish National Party',
             'title': 'Tommy Sheppard MP'},
  'question': {'idx': ['Q1.', 'Q1.1'],
               'member': ['If he will list his official engagements for '
                          'Wednesday 30 November. (902522)',
                          'In 2014, the Prime Minister’s predecessor David '
                          'Cameron signed up to the Smith commission, which '
                          'promised among other things that“nothing in this '
                          'report prevents Scotland becoming an independent '
                          'country…should the people of Scotland so '
                          'choose.”Does the Prime Minister share that view? If '
                          'he does, in the light of last week’s Supreme Court '
                          'judgme

In [571]:
# Inspect Indices
indices = [_["question"]["idx"] for _ in pmq_items]
indices

[['Q1.', 'Q1.1'],
 ['Q3.'],
 ['OP.1', 'OP.2', 'OP.3', 'OP.4', 'OP.5', 'OP.6'],
 ['Q4.'],
 ['OP.1', 'OP.2'],
 ['Q5.'],
 ['CAUGHT.1'],
 ['Q8.'],
 ['Q2.'],
 ['Q11.'],
 ['Q6.'],
 ['Q12.'],
 ['Q7.'],
 ['Q13.'],
 ['Q9.'],
 ['CAUGHT.2'],
 ['Q10.'],
 ['CAUGHT.3'],
 ['Q14.'],
 ['CAUGHT.4'],
 ['Q15.']]

In [572]:
# Output to JSON
import json

f_out = f"./data/pmqs_parsed_{DATE}.jsonl"
with open(f_out, "w") as f:
    for _ in pmq_items:
        f.write(json.dumps(_))
        f.write("\n")

In [573]:
# Write to Elastic DB
from elasticsearch import Elasticsearch
import configparser

# TODO: Read in config
config = configparser.ConfigParser()
config.read('elastic.ini')

es = Elasticsearch(
    config["ELASTIC"]["host"]
)

In [574]:
#es.indices.create(index="pmq")

In [575]:
# Add PMQ date to Schema
# Add Speaker Order indices to Schema
import logging

INDEX = "pmq"
for _ in pmq_items:
    doc = _
    resp = es.index(index=INDEX, body=doc, request_timeout=30)
    logging.info(resp['result'])

/var/folders/9y/jc1z3n7n78ndcq81djws05m00000gn/T/ipykernel_86772/3480264177.py:8: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  resp = es.index(index=INDEX, body=doc, request_timeout=30)
/var/folders/9y/jc1z3n7n78ndcq81djws05m00000gn/T/ipykernel_86772/3480264177.py:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.index(index=INDEX, body=doc, request_timeout=30)


In [ ]:
#         doc = {
#             "channel": channel,
#             "type": _["type"],
#             "message": _["text"],
#             "subtype": subtype,
#             "title": title,
#             "user": user,
#             "elements": elements,
#             "time_stamp": ts,
#             "date": date
#         }
#
#         created += 1
#         resp = es.index(index=INDEX, body=doc, request_timeout=30)
#         logging.info(resp['result'])

In [596]:
file = "/Users/joshua_sheppard/PycharmProjects/pmqBot/data/pmqs_parsed_2022-11-30.jsonl"
first = [json.loads(_) for _ in open(file)]
first_q = first[0]["question"]

first_q["member"]

['If he will list his official engagements for Wednesday 30 November. (902522)',
 'In 2014, the Prime Minister’s predecessor David Cameron signed up to the Smith commission, which promised among other things that“nothing in this report prevents Scotland becoming an independent country…should the people of Scotland so choose.”Does the Prime Minister share that view? If he does, in the light of last week’s Supreme Court judgment, will he bring forward legislation to allow that choice to be exercised?']

In [365]:
# TODO: Scale over other PMQs (3-5)
# TODO: .py Modularise ()
# TODO: Test Harness
# TODO: Usable Output
# TODO: Implement as Bot
# TODO: Store Objects (Elastic Search)
# TODO: Data Mine
# TODO: Handel Duplicates, ElasticSearch
# TODO: Use Parliament Hansard API

In [1]:
# import json
# import requests
# from pprint import pprint
#
# """ source: https://explore.data.parliament.uk/ """
# """ linked_data: http://lda.data.parliament.uk/answeredquestions """
#
# start = "2022-12-07"
# end = "2022-12-08"
#
# url = f"https://oralquestionsandmotions-api.parliament.uk/oralquestions/list?answeringDateStart={start}&answeringDateEnd={end}"
#
# r = requests.get(url, json=True)
# r.headers['content-type']
# r.encoding

'utf-8'

In [2]:
# response = r.json()
# pprint(response)

{'Errors': [],
 'PagingInfo': {'GlobalStatusCounts': [],
                'GlobalTotal': 65,
                'Skip': 0,
                'StatusCounts': [],
                'Take': 40,
                'Total': 65},
 'Response': [{'AnsweringBody': 'Prime Minister',
               'AnsweringBodyId': 23,
               'AnsweringMinister': {'Constituency': 'Richmond (Yorks)',
                                     'ListAs': 'Sunak, Rishi',
                                     'MnisId': 4483,
                                     'Name': 'Rishi Sunak',
                                     'Party': 'Conservative',
                                     'PartyColour': '0000ff',
                                     'PartyId': 4,
                                     'PhotoUrl': 'https://members-api.parliament.uk/api/Members/4483/Portrait?cropType=OneOne&webVersion=true',
                                     'PimsId': 6165,
                                     'Status': 'Active'},
               'Answ

In [ ]:
# def normalise_members(speakers):
#     members = []
#
#     for _ in speakers:
#         name = re.sub(r"\([^)]*\)", "", _)
#         name = re.sub(r"Dr", "", name)
#         name = re.sub(r"Mr", "", name)
#         name = re.sub(r"Mrs", "", name)
#         name = re.sub(r"Sir", "", name)
#         name = re.sub(r"Dame", "", name)
#         name = re.sub(r"[^\x00-\x7F]+", "", name)
#
#         members.append(name.strip())
#
#     return members
#
# def normalise_member(member):
#
#     member = re.sub(r"\([^)]*\)", "", member)
#     member = re.sub(r"Dr", "", member)
#     member = re.sub(r"Mr", "", member)
#     member = re.sub(r"Mrs", "", member)
#     member = re.sub(r"Dame", "", member)
#     member = re.sub(r"[^\x00-\x7F]+", "", member)
#
#     return member.strip()

# def get_member(name="Rishi Sunak"):
#     name = name.split()
#     print(name)
#
#     members_api = "https://members-api.parliament.uk/api/Members/Search?"
#     if len(name) > 1:
#         first = name[0]
#         last = name[1]
#         params = f"Name={first}%20{last}"
#
#     else:
#         name = name[-1]
#         params = f"Name={name}"
#
#     url = members_api + params
#
#     try:
#         result = requests.get(url, json=True).json()
#
#     except:
#         return None
#
#     if len(result["items"]) > 0:
#         id_ = result["items"][0]["value"]["id"]
#         full_title = result["items"][0]["value"]["nameFullTitle"]
#         party = result["items"][0]["value"]["latestParty"]["name"]
#         img = result["items"][0]["value"]["thumbnailUrl"]
#
#         return id_, full_title, party, img
#
#     else: return result["items"]

# for _ in speakers:
#     count += 1
#     if _ == "Prime Minister":
#         member_profiles.append(get_member())
#
#     else:
#         norm = normalise_member(_)
#         member_profiles.append(get_member(norm))

In [9]:
# # Get Member
#
# def get_post(member_id="4483"):
#     member_post = f"https://members-api.parliament.uk/api/Members/{member_id}/Biography"
#     result = requests.get(member_post, json=True).json()
#
#     if result["value"]["governmentPosts"][0] != [""]:
#         post = result["value"]["governmentPosts"]
#
#     if result["value"]["oppositionPosts"][0] != [""]:
#         post = result["value"]["oppositionPosts"]
#
#     return post
#
#
# # TODO: Role ie Opposition vs Member
# def extract_member(name):
#     #name = debate_item[0]
#     print(name)
#
#     if name.startswith("Prime Minister"):
#         id_, title, party, img = get_member()
#         post = get_post()
#
#     else:
#         name_split = name.split()
#         first, last = name_split[0], name_split[1]
#         id_, title, party, img = get_member(first, last)
#         post = get_post(id_)
#
#     return id_, title, party, post, img

In [191]:
# Comprehensive PMQs Object
#
# class DebateItem:
#     def __init__(self):
#         self.member = {}
#         self.question = {
#             "idx": []
#         }
#         self.response = {}
#
# debate_item = DebateItem().__dict__
#
# class PMQ:
#     def __init__(self, type):
#         self.type = type
#
#     def __call__(self, debate_item):
#         if self.type == "opposition":
#             debate_item["question"]["opposition_opening"] = []
#             debate_item["question"]["supplement"] = []
#
#             debate_item["response"]["opposition_opening"] = []
#             debate_item["response"]["supplement"] = []
#
#             return debate_item
#
#         elif self.type == "supplement":
#             debate_item["question"]["tabled"] = []
#             debate_item["question"]["supplement"] = []
#
#             debate_item["response"]["tabled"] = []
#             debate_item["response"]["supplement"] = []
#
#             return debate_item
#
#         elif self.type == "member":
#             debate_item["question"]["tabled"] = []
#             debate_item["response"]["tabled"] = []
#
#             return debate_item
#
#         else:
#             raise ValueError("incorrect type")
#
# pmq = PMQ(type="member")(debate_item)
# pmq

{'member': {},
 'question': {'idx': [], 'tabled': []},
 'response': {'tabled': []}}

In [303]:
# Comprehensive PMQs Object
#
# import re
#
# debate_chunks = []
# for _ in boundaries:
#     seek, stop = _[0], _[1]+1
#     debate_chunks.append((seek, stop, pmqs[seek: stop]))
#
# def get_type(member, questions):
#     member = member.lower()
#
#     if member.startswith("keir") or member.startswith("stephen flynn"):
#         return "opposition"
#
#     elif len(questions) > 1:
#         return "supplement"
#
#     else: return "member"
#
# debate = (_ for _ in debate_chunks)
# pmq_items = []
#
# counta = 0
# while counta < (len(debate_chunks)):
#
#     seek, stop, discourse = next(debate)
#     member = discourse[0][0]
#
#     # TODO: Make 'Prime Minister' Robust
#     questions = [_ for _ in discourse if _[0] != "Prime Minister"]
#     responses = [_ for _ in discourse if _[0] == "Prime Minister"]
#
#     type = get_type(member, questions)
#
#     debate_item = DebateItem().__dict__
#     pmq = PMQ(type=type)(debate_item)
#
#     for idx, _ in enumerate(zip(questions, responses)):
#         question, response = _
#
#         member = question[0]
#         q = question[1]
#         r = response[1]
#
#         q_idx = re.findall(r"(\b[Q]{1}[0-9]+[.])", q)
#         q_idx = (q_idx[0]) if q_idx != [] else ""
#
#         if idx == 0:
#             pmq["member"] = member
#
#         if type == "supplement":
#             if idx > 0:
#                 lst_idx = pmq["question"]["idx"][-1]
#                 q_idx = f"{lst_idx}{idx}"
#
#                 pmq["question"]["idx"].append(q_idx)
#                 pmq["question"]["supplement"].append(q)
#                 pmq["response"]["supplement"].append(r)
#
#             else:
#                 pmq["question"]["idx"].append(q_idx)
#                 pmq["question"]["tabled"].append(q)
#                 pmq["response"]["tabled"].append(r)
#
#         elif type == "opposition":
#             if idx > 0:
#                 pmq["question"]["supplement"].append(q)
#                 pmq["response"]["supplement"].append(r)
#
#             else:
#                 pmq["question"]["opposition_opening"].append(q)
#                 pmq["response"]["opposition_opening"].append(r)
#
#         elif type == "member":
#             pmq["question"]["idx"].append(q_idx)
#             pmq["question"]["tabled"].append(q)
#             pmq["response"]["tabled"].append(r)
#
#     pmq_items.append(pmq)
#     counta += 1

In [ ]:

    # if cur_member != "prime_minister":
    #     debate_item = {}
    #     debate_item["member"] = cur_member
    #     debate_item["questions"]["tabled"] = get_question()
    #     _ += 1
    #     next_member()




    # TODO: affirmative
    # if cur_member != "prime_minister":
    #     debate_item = {}
    #     debate_item["member"] = cur_member
    #     debate_item["questions"]["tabled"] = get_question()
    #     _ += 1
    #     next_member()
    #
    # if cur_member == "prime_minister":
    #     debate_item["response"]["tabled"] = get_question()
    #     _ += 1
    #
    # if cur_member == previous_member:
    #     debate_item["questions"]["supplement"] = get_question()
    #     _ += 1
    #
    # if counta == 3:
    #     break

In [15]:
# def cur_member():
#     global cur_member
#     global previous_member
#
#     member = pmqs[_][0]
#
#     if member.startswith("Prime Minister"):
#         cur_member = "prime_minister"
#
#     else:
#         cur_member = member
#         history.append(member)
#         previous_member = history[-2] if len(history) > 1 else history[-1]

# def get_question():
#     question = pmqs[_][1]
#
#     return question

# def is_supplemented():
#     if cur_member == previous_member:
#         return True
#
#     else:
#         return False

# def next_debate():
#     boundary = next(debate)
#
#     seek, finish = boundary[0], boundary[1]
#
#     return (seek, pmqs[seek: finish])

In [159]:
# history = []
# pm = False
#
# cur_speaker = ""
# previous_speaker = ""
#
# pm = False
#
# question_type = []
# question_idx = []
# question_history = []
#
# opposition_counts = {
#     "OPPOSITION_LABOUR": 0,
#     "OPPOSITION_SNP": 0,
#     "OPPOSITION_LIBERAL_DEMOCRATS": 0,
# }
# _ = 0
#
# debate = []
#
# class Question:
#      def __init__(self):
#         self.idx = ""
#         self.asking_member = ""
#         self.question = ""
#         self.response = ""
#         self.supplement = {
#             "idx": "",
#             "supplement": "",
#             "response": "",
#          }
#
# def turn():
#     global debate_item
#     global cur_speaker
#     global previous_speaker
#     global question_type
#     global _
#     global pm
#     global question
#     global debate
#     global question
#     global member
#
#     debate_item = pmqs[_]
#     id_, title, party, post, img = extract_member(debate_item)
#
#     # TODO: Class Obj
#     member = {
#         "id_": id_,
#         "title": title,
#         "party": party,
#         "post": post,
#         "img": img,
#     }
#
#     if member["title"].startswith("Rt Hon Rishi"):
#         pm = True
#
#     else: pm = False
#
#     _ += 1

In [ ]:
# question_types = []
#
# def type():
#     if previous_speaker and not pm:
#         if cur_speaker == previous_speaker:
#             if turns < 2:
#                 question_types.append("OPENING")
#
#             elif "leader" in cur_speaker["post"].lower().split():
#                 question_types.append("OPPOSITION")
#
#             else:
#                 question_types.append("SUPPLEMENT")
#
#         elif cur_speaker != previous_speaker:
#             if "leader" in cur_speaker["post"].lower().split():
#                 question_types.append("OPPOSITION")
#
#             else:
#                 question_types.append("MEMBER")
#
# engagements = []
# turns = 0
# _ = 0
#
# turn()
# while _ < (len(pmqs) - 1):
#     turn()
#
#     # Create question
#     turns += 1
#
#     if turns > 2:
#         break
#
#     if not pm:
#         if cur_type == "SUPPLEMENT":
#             question_count = 2
#             previous_question = debate[-1]
#
#             idx = f"{idx}{question_count}"
#
#             previous_question["supplement"]["idx"] = idx
#             previous_question["supplement"]["supplement"] = debate_item[1]
#
#         else:
#             question = Question().__dict__
#             debate.append(question)
#
#             type()
#             cur_type = question_types[-1]
#             print(cur_type)
#
#             idx = re.findall(r"(\b[Q]{1}[0-9]+[.])", cur_question[1])
#             idx = str(idx[0]) if idx != [] else ""
#
#
#
#         question_count = 0
#         question["asking_member"] = member.copy()
#
#         idx = re.findall(r"(\b[Q]{1}[0-9]+[.])", debate_item[1])
#         idx = str(idx[0]) if idx != [] else ""
#
#         question["idx"] = idx
#         question["question"] = debate_item[1]
#         debate.append(question)
#
#
#         previous_speaker = debate[-2]["asking_member"] if len(debate) > 1 else debate[-1]
#
#     elif pm:
#         if cur_type == "SUPPLEMENT":
#             print("SUPPLEMENT")
#             cur_question["supplement"]["response"] = debate_item[1]
#             print(debate_item[1])
#             print(cur_question)
#
#         elif cur_type in ["MEMBER", "OPENING", "OPPOSITION"]:
#             cur_question["response"] = debate_item[1]
#
#     cur_speaker = debate[-1]["asking_member"]
#     cur_question = debate[-1]
#
#
#     # if cur_type in ["MEMBER", "OPENING", "SUPPLEMENT"]:
#     #     if cur_type == "SUPPLEMENT":
#     #         question_count += 1
#     #         previous_question = history[-1]
#     #
#     #         idx = f"{idx}{question_count}"
#     #
#     #         previous_question["supplement"]["idx"] = idx
#     #         previous_question["supplement"]["supplement"] = debate_item[1]
#
#
#     # if peek == "OPPOSITION":
#     #     question = {
#     #         "idx": "",
#     #         "asking_member": "",
#     #         "questions": [],
#     #         "responses": []
#     #     }
#     #
#     #     idx = f"OP{opposition}.{opp_count}"
#     #     question_idx.append(idx)
#     #
#     #     question["idx"] = idx
#     #     question["asking_member"] = cur_speaker
#     #     question["questions"].append(debate)
#
# class Debate_item():
#     def __init__(self):
#         self.member = {}
#         self.questions = {
#             "tabled": "",
#             "supplement": []
#         }
#         self.response = {
#             "tabled": "",
#             "supplement": []
#         }
#
# class Debate_item():
#     member = {}
#     questions = {
#         "tabled": "",
#         "supplement": []
#     }
#     response = {
#         "tabled": "",
#         "supplement": []
#     }
#
# item_ = Debate_item().__dict__

In [ ]:
# engagements = []
# turns = 0
# _ = 0
#
# turn()
# while _ < (len(pmqs) - 1):
#
#     peek = question_type[-1]
#
#     if not pm:
#         turns += 1
#
#     if pm:
#         if peek == "SUPPLEMENT":
#             question["supplement"]["response"] = debate_item[1]
#             continue
#
#         elif peek == "MEMBER" or peek == "OPPOSITION":
#             question["response"].append(debate_item[1])
#             continue
#
#     if peek != "OPPOSITION":
#         if peek != "SUPPLEMENT":
#             question_count = 0
#
#             question = {
#                 "idx": "",
#                 "asking_member": "",
#                 "question": [],
#                 "response": [],
#                 "supplement": {
#                         "idx": "",
#                         "supplement": "",
#                         "response": "",
#                     },
#             }
#
#         idx = re.findall(r"(\b[Q]{1}[0-9]+[.])", debate_item[1])
#         idx = str(idx[0]) if idx != [] else ""
#
#         question["idx"] = idx
#         question["asking_member"] = cur_speaker["title"]
#         question["question"].append(debate_item[1])
#
#         question_idx.append(idx)
#
#         if peek == "SUPPLEMENT":
#             # Extend existing question
#             question_count += 1
#             idx = f"{idx}{question_count}"
#
#             question["supplement"]["idx"] = idx
#             question["supplement"]["supplement"] = debate_item[1]
#
#     engagements.append(question)
#     turn()
#     # if peek == "OPPOSITION":
#     #     question = {
#     #         "idx": "",
#     #         "asking_member": "",
#     #         "questions": [],
#     #         "responses": []
#     #     }
#     #
#     #     idx = f"OP{opposition}.{opp_count}"
#     #     question_idx.append(idx)
#     #
#     #     question["idx"] = idx
#     #     question["asking_member"] = cur_speaker
#     #     question["questions"].append(debate)

In [ ]:

    # # TODO: Robust over multiple Oppositions
    # # TODO: Update 'or' operation
    # elif
    #     opposition = int(position(cur_speaker)[-1])
    #     opposition_party = opposition_idx[opposition]
    #
    #     opposition_counts[opposition_party] += 1
    #
    #     question = {
    #         "idx": "",
    #         "asking_member": "",
    #         "questions": [],
    #         "supplements": {
    #                 "idx": "",
    #                 "supplement": "",
    #                 "responses": "",
    #             },
    #         "responses": []
    #     }
    #
    #     idx = f"OP{opposition}.{opp_count}"
    #     question_idx.append(idx)
    #
    #     question["idx"] = idx
    #     question["asking_member"] = cur_speaker
    #     question["questions"].append(debate)

In [15]:
# pmqs = soup.find_all("p", class_="Question")
# debate = soup.find_all("div", class_="debate-item debate-item-contributiondebateitem")
#
# child_debates = soup.find_all("div", class_="child-debate")
# contributions = soup.find_all("div", class_="contribution")
#
# content = soup.find_all("div", class_="content")
# print(content)

In [16]:
# SIMPLE
# _ = 0
# debate_it = 0
# debating = True
# debate_items = []
# while debating and (_ < len(pmqs) - 1):
#     responding = False
#
#     debate_item = []
#
#     debate_it += 1
#     print(f"Debate {debate}")
#
#     member, debate = pmqs[_]
#     debate_item.append(pmqs[_])
#
#     print(member, "\n")
#     print(debate, "\n")
#
#     lookahead = pmqs[_ + 1]
#
#     if lookahead[0] == "Prime Minister":
#         responding = True
#
#         while responding and (_ < len(pmqs) - 1):
#             _ += 1
#
#             if pmqs[_][0] != "Prime Minister":
#                 responding = False
#
#             else:
#                 print("PM: ")
#                 response = pmqs[_][1]
#                 debate_item.append(pmqs[_])
#                 print(response)
#
#     debate_items.append(debate_item)
#     print("====")

In [17]:
# WITH CASES

# QUESTIONING SEQUENCE
# ===
#
# if first_question is asked by an goverment_mp:
#     next_question(leader_of_opposition)
#
# if the first_question is asked by an opposition_mp:
#     next_question(govenrment_member)
#
# # TODOs: Military death
#
# gender = "he" if member == male else "she"
# speak(emergency=False, init=False, block=False):
#     if init=True:
#         if emergency:
#             print(f"emergent policy {emergent_topic}")
#
#         else:
#             print(f"if he {gender} will list his {gender} engagements for the day"
#
#     if (leader_of_opposition and block):
#         print([i for i in question_num])
#
#     elsif (leader_of_opposition and not block):
#         print([i for i in question_num[0:len(question_num//2)]
#         pm_turn()
#         print([i for i in question_num[len(question_num//2):]
#
#     else:
#         print(question)
#
#
# question_counta = 0
# next_question():
#     question_counta += 1
#         if emergency:
#             speak(emergency=True)
#
#         else:
#             speak(init=True)
#
#     if len(questions) > 15:
#         break
#
#     if question_counta == 1:
#         speak(init=True)
#
#     if leader_of_opposition == offical:
#         table_in_advance = False
#         question_num = set(6)
#
#         if block:
#             speak(block=True)
#
#         else:
#             speak(block=False)
#
#     elif leader_of_opposition == second_official:
#         table_in_advance = False
#         question_num = set(2)
#         speak()
#
#     elif leader_of_opposition == third_official:
#         table_in_advance = False
#         question_num = set(2)
#         speak()
#
#     else:
#         table_in_advance = True
#         questions = set(1)
#         return questions
#         speak()
#
# """

In [15]:
# # TODO: Construct through Parliament API
# # TODO: PMQs as a Class
#
# members = {
#     "Prime Minister": "head of government",
#     "Rishi Sunak": "head of government",
#     "Keir Starmer": "leader of the opposition 1",
#     "Stephen Flynn": "leader of the opposition 2",
#     "Dr Philippa Whitford": "SNP",
#     "Anthony Browne": "Conservative",
#     "Nickie Aiken": "Conservative",
#     "David Morris": "Conservative",
#     "Stephen Farry": "Alliance",
#     "Darren Henry": "Conservative",
#     "Ian Byre": "Labour",
#     "Gareth Davies": "Conservative",
#     "Siobhan McDonagh": "Labour",
#     "Rehman Chishti": "Labour",
#     "Mary Kelly Fox": "Labour",
#     "Conor Burns": "Conservative",
#     "Allan Dorans": "SNP",
#     "Jackie Doyle-Price": "Conservative",
#     "Dame Diana Johnson": "Labour",
#     "Andrew Brigden": "Conservative",
#     "Kevin Brennon": "Conservative",
#     "Matt Warman": "Conservative",
#     "Chris Elmore": "Labour",
#     "Mr Robin Walker": "Conservative",
#     "Clair Hanna": "SDLP",
# }

In [ ]:
"""
PMQs Algo
===

PMQs - the most distinctive and internationally famous feature of British politics ...

1. backbench members proposing a question enter names on an order paper

2. name of entrants is shuffled in a ballot to product random order

3. speaker calls members as a random series, alternating government and opposition positions

4. if member == leader of the 'official' opposition:
    ask 6 questions
        questions == set(6) or (set(3) + set(3))

4. non chosen members may be asked to provide a supplementatry question according to 'catch the eye ()' of the speaker

QUESTIONING SEQUENCE
===

if first_question is asked by an goverment_mp:
    next_question(leader_of_opposition)

if the first_question is asked by an opposition_mp:
    next_question(govenrment_member)

# TODOs: Military death

gender = "he" if member == male else "she"
speak(emergency=False, init=False, block=False):
    if init=True:
        if emergency:
            print(f"emergent policy {emergent_topic}")

        else:
            print(f"if he {gender} will list his {gender} engagements for the day"

    if (leader_of_opposition and block):
        print([i for i in question_num])

    elsif (leader_of_opposition and not block):
        print([i for i in question_num[0:len(question_num//2)]
        pm_turn()
        print([i for i in question_num[len(question_num//2):]

    else:
        print(question)


question_counta = 0
next_question():
    question_counta += 1
        if emergency:
            speak(emergency=True)

        else:
            speak(init=True)

    if len(questions) > 15:
        break

    if question_counta == 1:
        speak(init=True)

    if leader_of_opposition == offical:
        table_in_advance = False
        question_num = set(6)

        if block:
            speak(block=True)

        else:
            speak(block=False)

    elif leader_of_opposition == second_official:
        table_in_advance = False
        question_num = set(2)
        speak()

    elif leader_of_opposition == third_official:
        table_in_advance = False
        question_num = set(2)
        speak()

    else:
        table_in_advance = True
        questions = set(1)
        return questions
        speak()

"""